Hier is ons fantastische huiswerk

# Step 1: Simulate Rankings of Relevance for E and P (5 points)
In the first step you will generate pairs of rankings of relevance, for the production P and experimental E, respectively, for a hypothetical query q. Assume a 3-graded relevance, i.e. {N, R, HR}. Construct all possible P and E ranking pairs of length 5. This step should give you about.

In [68]:
import itertools
import math

possible_rankings = []

possible_evals = 3

for i in range(possible_evals):
    for j in range(possible_evals):
        for k in range(possible_evals):
            for l in range(possible_evals):
                for m in range(possible_evals):
                    new_list = [i, j, k, l, m]
                    possible_rankings.append(new_list)

p_rankings = possible_rankings
e_rankings = possible_rankings

possible_combinations = []
for comb in itertools.combinations(possible_rankings, 2):
    possible_combinations.append(comb)

# Step 2: Implement Evaluation Measures (10 points)
Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above. 
(Note 2: Some of the aforementioned measures require the total number of relevant and highly relevant documents in the entire collection – pay extra attention on how to find this)

In [69]:
def precision(ranking, rank):
    relevant = 0
    for i in range(rank):
        if ranking[i] > 0:
            relevant += 1
    return relevant /rank

def NDCG(ranking, rank):
    total_dcg = 0
    total_ideal_dcg = 0

    for i in range(rank):
        total_dcg += ((2**ranking[i])-1)/math.log2(1+i+1)
        ideal_ranking = sorted(ranking, reverse=True)
        total_ideal_dcg += ((2**ideal_ranking[i])-1)/math.log2(1+i+1)
    
    try:
        result = total_dcg/total_ideal_dcg
    except:
        result = 0
    return result

def ERR(ranking, rank):
    p = 1
    ERR = 0
    for i in range(rank):
        relevant = relevance_map(ranking[i])
        ERR += p * relevant / (i+1)
        p = p * (1-relevant)
    return ERR

def relevance_map(relevant):
    max_relevant = 2
    return (2**relevant - 1)/2**max_relevant
    

In [70]:
p_precision = precision(p_rankings[10], 5)
print(p_precision)

p_ndcg = NDCG(p_rankings[10], 5)
print(p_ndcg)

p_err = ERR(p_rankings[10], 5)
print(p_err)

test = ([0, 0, 0, 2, 2], [0, 0, 1, 0, 1])

p_precision = precision(test[0], 5)
e_precision = precision(test[1], 5)

p_ndcg = NDCG(test[0], 5)
e_ndcg = NDCG(test[1], 5)

p_err = ERR(test[0], 5)
e_err = ERR(test[1], 5)

print(p_precision, p_ndcg, p_err)
print(e_precision, e_ndcg, e_err)
                



0.4
0.5437713091520254
0.12083333333333332
0.4 0.5012658353418872 0.225
0.4 0.5437713091520254 0.12083333333333332


# Step 3: Calculate the 𝛥measure (0 points)
For the three measures and all P and E ranking pairs constructed above calculate the difference: 𝛥measure = measureE-measureP. Consider only those pairs for which E outperforms P.


In [71]:
good_combinations = []

for comb in possible_combinations:
    p_precision = precision(comb[0], 5)
    e_precision = precision(comb[1], 5)
    
    p_ndcg = NDCG(comb[0], 5)
    e_ndcg = NDCG(comb[1], 5)
      
    p_err = ERR(comb[0], 5)
    e_err = ERR(comb[1], 5)
    
    p_total = p_precision + p_ndcg + p_err
    e_total = e_precision + e_ndcg + e_err
    
    if(e_total > p_total):
        good_combinations.append(comb)
        
print(len(good_combinations))

26576


# Step 4: Implement Interleaving (15 points)
Implement 2 interleaving algorithms: (1) Team-Draft Interleaving OR Balanced Interleaving, AND (2), Probabilistic Interleaving. The interleaving algorithms should (a) given two rankings of relevance interleave them into a single ranking, and (b) given the users clicks on the interleaved ranking assign credit to the algorithms that produced the rankings.

(Note 4: Note here that as opposed to a normal interleaving experiment where rankings consists of urls or docids, in our case the rankings consist of relevance labels. Hence in this case (a) you will assume that E and P return different documents, (b) the interleaved ranking will also be a ranking of labels.)


In [80]:
import random

def balanced_interleave(P, E):
    Kp = 0
    Ke = 0
    result = []
    p_firsrt = False
    coin = random.uniform(0,1)
    if coin > .5:
        p_first = True
    
    while Kp < len(P) or Ke < len(E):
        if Kp < Ke or (Kp == Ke and p_first):
            result.append({"rel": P[Kp], "source": "P"})
            Kp += 1
        else:
            result.append({"rel": E[Ke], "source": "E"})
    return result
            
            

In [81]:
for comb in good_combinations:
    print(comb)
    result = balanceinterleave(comb[0], comb[1])
    rankings.append(result)
    

([], [])
([], [0, 0, 0])


IndexError: list assignment index out of range

# Step 5: Implement User Clicks Simulation (15 points)
Having interleaved all the ranking pairs an online experiment could be ran. However, given that we do not have any users (and the entire homework is a big simulation) we will simulate user clicks.
We have considered a number of click models including:<br />
- Random Click Model (RCM)<br />
- Position-Based Model (PBM)<br />
- Simple Dependent Click Model (SDCM)<br />
- Simple Dynamic Bayesian Network (SDBN)<br />

Consider two different click models, (a) the Random Click Model (RCM), and (b) one out of the remaining 3 aforementioned models. The parameters of some of these models can be estimated using the Maximum Likelihood Estimation (MLE) method, while others require using the Expectation-Maximization (EM) method. Implement the two models so that (a) there is a method that learns the parameters of the model given a set of training data, (b) there is a method that predicts the click probability given a ranked list of relevance labels, (c) there is a method that decides - stochastically - whether a document is clicked based on these probabilities.<br />

Having implemented the two click models, estimate the model parameters using the Yandex Click Log 


In [5]:
from collections import Counter

class RCM:
    
    def __init__(self):
        self.p = 0
    
    def train(self, data):
        #p = #clicks / #documents
        documents = 0
        clicks = 0
        for row in data:
            line = row.split()
            if line[2] == "Q":
                documents += len(line)-5
            else:
                clicks += 1
        self.p = clicks/documents
                

class PBM:
    def __init__(self):
        self.p = dict()
        
    def train(self, data):
        documents = Counter()
        clicks = Counter()
        
        for row in data:
            line = row.split()
            if line[2] == "Q":
                cur_urls = dict()
                for i in range(0, len(line)-5):
                    cur_urls[i] = line[i+5]
                    documents[i] += 1
            else:
                url_click = line[3]
                url_position = 0
                for pos, url in cur_urls.items():
                    if url_click == url:
                        url_poistion = pos
                        break
                clicks[pos] += 1
        for pos, document_count in documents.items():
            self.p[pos] = clicks[pos]/document_count

In [7]:
random_click_model = RCM()
position_click_model = PBM()

with open('YandexRelPredChallenge.txt') as f:
    random_click_model.train(f)

with open('YandexRelPredChallenge.txt') as f:
    position_click_model.train(f)


# Step 6: Simulate Interleaving Experiment (10 points)
Having implemented the click models, it is time to run the simulated experiment.
For each of interleaved ranking run N simulations for each one of the click models implemented and measure the proportion p of wins for E.

(Note 7: Some of the models above include an attractiveness parameter 𝑎uq. Use the relevance label to assign this parameter by setting 𝑎uq for a document u in the ranked list accordingly. (See Click Models for Web Search)

In [11]:
def simulate_random_model(rankings, N):
    total = N * len(rankings)
    e_wins = 0
    for i in range(N):
        for url in rankings:
            coin = random.uniform(0,1)
            if coin < random_click_model and url.source == "e":
                e_wins += 1
    return e_wins / total

def simulate_position_model(rankings, N):
    total = N * len(rankings)
    e_wins = 0
    for i in range(N):
        for j in range(len(rankings)):
            coin = random.uniform(0,1)
            if coin < position_click_model.p[j]*rankings[j].relevance and rankings[j].source == "e":
                e_wins += 1
    return e_wins / total

# Step 7: Results and Analysis (30 points)
Compare the results of the offline experiments (i.e. the values of the 𝛥measure) with the results of the online experiment (i.e. proportion of wins), analyze them and reach your conclusions regarding their agreement.
Use easy to read and comprehend visuals to demonstrate the results;
Analyze the results on the basis of:
- the evaluation measure used,
- the interleaving method used,
- the click model used.

Report and ground your conclusions.

(Note 8: This is the place where you need to demonstrate your deeper understanding of what you have implemented so far; hence the large number of points assigned. Make sure you clearly do that so that the examiner of your work can grade it accordingly.)
